# Analysis of NewAthena Community

In [ ]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import re
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.path import Path
from matplotlib.patches import PathPatch
from matplotlib.font_manager import FontProperties
import matplotlib.patheffects as pe

## Read data and prepare table for analysis

### Import Data file with registrations 


In [ ]:
#regis_file = "registrations_final_20250416.xlsx"
date_data = "20251114"
save_figs = True
# change dir to the date_data folder
os.chdir(f"/home/ceballos/INSTRUMEN/ATHENA/ACO/Community/{date_data}")
regis_file = f"registrations_final_{date_data}.xlsx"
# get date from filename
date = re.search(r"_(\d{8})", regis_file)
if date:
    date = date.group(1)
print(f"Using date: {date}")
cleanregis_file = "cleaned_registrations.xlsx"
# Load data from xlsx file:first row is header
df = pd.read_excel(regis_file, header=0)

#### Remove duplicates and save to a clean file

In [ ]:
# Remove duplicates based on a specific column
df_cleaned_column = df.drop_duplicates(subset=['Name'], keep='first') 

# Save the cleaned data to a new Excel file
df_cleaned_column.to_excel(cleanregis_file, index=False)

### Reading the cleaned_registrations file

In [ ]:
# Load data from xlsx file:first row is header
df = pd.read_excel(cleanregis_file, header=0)
# convert NaN to empty string but keep boolean values
df = df.where(pd.notnull(df), '')
# fill NaN with empty string
df = df.fillna('')
df

### Reduce Title of columns: df_clean0

In [ ]:
# print names of columns
print(df.columns)
df_clean0 = df.copy()
# change column names:
# 'INSTEAD, I'd like to be an ACTIVE MEMBER at:' -> 'ACTIVE_MEMBER'
df_clean0.rename(columns={"INSTEAD, I'd like to be an ACTIVE MEMBER at:": "ACTIVE_MEMBER"}, inplace=True)
# 'IN ADDITION, I'd like to be CHAIR at:' -> 'CHAIR_REQUESTED'
df_clean0.rename(columns={"IN ADDITION, I'd like to be CHAIR at:": "CHAIR_REQUESTED"}, inplace=True)
# 'I'd prefer being only a SUPPORT MEMBER' -> 'SUPPORT_MEMBER'
df_clean0.rename(columns={"I'd prefer being only a SUPPORT MEMBER": "SUPPORT_MEMBER"}, inplace=True)
# email address
df_clean0.rename(columns={"Email Address": "EMAIL"}, inplace=True)
print(df_clean0.columns)


### Simplify WG labels

In [ ]:
# replace strings with WG numbers and WG names as only WG numbers (taking into account that several WGs can be listed in the same cell)
# for example: "WG4: Compact objects" -> "WG4"
df_clean = df_clean0.copy()
df_clean["ACTIVE_MEMBER"] = df_clean0["ACTIVE_MEMBER"].apply(lambda x: ";".join(re.findall(r'\b(WG\d+)\b', x)))
df_clean["CHAIR_REQUESTED"] = df_clean0["CHAIR_REQUESTED"].apply(lambda x: ";".join(re.findall(r'\b(WG\d+)\b', x)))
df_clean

### Get subtables

In [ ]:
df_support = df_clean[df_clean["SUPPORT_MEMBER"] == "Yes"]
df_chair_req = df_clean[df_clean["CHAIR_REQUESTED"] != ""]
df_active = df_clean[df_clean["ACTIVE_MEMBER"] != ""]

## Plot distribution of Members types (Support/Active/Chairs) by country 

In [ ]:
# get total number of support members
total_support = len(df_support)
total_members = len(df_clean)
total_chairs_req = len(df_chair_req)
total_active = len(df_active)

# Get total members per country
all_countries = df_clean["Country1"].value_counts()

# Sort countries by total members (descending)
sorted_countries = all_countries.sort_values(ascending=False).index.tolist()


# Build aligned counts
total = df_clean["Country1"].value_counts().reindex(sorted_countries, fill_value=0)
active = df_active["Country1"].value_counts().reindex(sorted_countries, fill_value=0)
support = df_support["Country1"].value_counts().reindex(sorted_countries, fill_value=0)
chair_req = df_chair_req["Country1"].value_counts().reindex(sorted_countries, fill_value=0)

# Combine into one DataFrame
counts_df = pd.DataFrame({
    f'Total members ({total_members})': total,
    f'Active members ({total_active})': active,
    f'Support members ({total_support})': support,
    f'Requested Chair ({total_chairs_req})': chair_req
})

bars_colors = ['red', 'purple', 'blue', 'green']

# Plot and add label to each barplot
ax = counts_df.plot(kind='bar', width=0.8, logy=True, figsize=(12, 6), color=bars_colors, alpha=0.5)
#ax = counts_df.plot(kind='bar', width=0.8, logy=False, figsize=(12, 6), color=bars_colors, alpha=0.5)
ax.set_xlabel("")
ax.set_ylabel("Number of Members")
ax.set_title("NASC Membership by Country")
plt.xticks(rotation=90)
plt.tight_layout()

# Save the plot to a file
if save_figs:
    plt.savefig("NASC_Membership_by_Country.png", dpi=300, bbox_inches='tight')
    #plt.savefig("NASC_Membership_by_Country_nolog.png", dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
#percentage of community members in each country
country_percentage_total = (total / total_members) * 100
country_percentage_active = (active / total_active) * 100
# sort by percentage
country_percentage_total = country_percentage_total.sort_values(ascending=False)
country_percentage_active = country_percentage_active.sort_values(ascending=False)
# save to csv
country_percentage_total.to_csv("NASC_Membership_by_Country_percentage.csv", header=["Percentage"])
country_percentage_active.to_csv("NASC_Membership_by_Country_active_percentage.csv", header=["Percentage"])

#### Check cell

In [ ]:
country = "United Kingdom"
affil = ""
df_country = df_clean[df_clean["Country1"] == country]
if affil:
    df_filtered =df_filtered = df_country[df_country["Affiliation1"].str.contains(affil, case=False, na=False)]
else:
    df_filtered = df_country.copy()
#print members from "country" in blocks of ten members
#for i in range(0, len(df_country), 10):
#    print(df_country.iloc[i:i+10][["Name", "Country1", "ACTIVE_MEMBER", "SUPPORT_MEMBER","CHAIR_REQUESTED"]])

#print(f"Members from {country}:")
#print(df_country[["Name", "Country1", "ACTIVE_MEMBER", "CHAIR_REQUESTED"]])
print(f"Total members from {country}: {len(df_country)}")
# print chairs (requested) from country
#print("Requested Chairs from {country}:")
#print(df_country[df_country["CHAIR_REQUESTED"] != ""])
print(f"Total requested chairs from {country}: {len(df_country[df_country['CHAIR_REQUESTED'] != ''])}")
# print active members of country
print(f"Total active members from {country}: {len(df_country[df_country['ACTIVE_MEMBER'] != ''])}")
# print support members of country
print(f"Total support members from {country}: {len(df_country[df_country['SUPPORT_MEMBER'] == 'Yes'])}")

# print member from country who applied for chair but not activer nor support
print(f"Total members from {country} who applied for chair but not active nor support: {len(df_country[(df_country['CHAIR_REQUESTED'] != '') & (df_country['ACTIVE_MEMBER'] == '') & (df_country['SUPPORT_MEMBER'] == 'No')])}")
# print which members from country did not apply for active nor support
print(f"Total members from {country} who did not apply for active nor support: {len(df_country[(df_country['ACTIVE_MEMBER'] == '') & (df_country['SUPPORT_MEMBER'] == 'No')])}")

# save members from filtered list to a file: Name, Affiliation1, Country1
df_filtered = df_filtered[["Name", "Affiliation1", "Country1", "EMAIL"]]
if affil:
    file_country = f"members_from_{country.replace(' ', '_')}_{affil.replace(' ', '_')}.xlsx"
else:
    file_country = f"members_from_{country.replace(' ', '_')}.xlsx"
df_filtered.to_excel(file_country, index=False)  
print(df_filtered)

### Pie chart for country distribution

In [ ]:

# === Active Membership by Country — Full Pie + Subtle Per-Wedge Shadow (Fix B)
#     + Counts inside + Straight Elbow Leaders + Dot on Wedge
membership_toplot = "all"
if membership_toplot == "active":
    df_source = df_active
    title       = "Active Membership by Country"
elif membership_toplot == "support":
    df_source = df_support
    title       = "Support Membership by Country"      
elif membership_toplot == "all":
    df_source = df_clean
    title       = "All Membership by Country"

# ---- Source & columns ----
country_col = "Country1"

# ---- Labeling rule (choose Top-N or min %)
top_n       = 10                 # labels for Top N countries
min_percent = None               # or set e.g., 3.0 and set top_n=None
group_other = True

# ---- Build distribution ----
counts = (
    df_source[country_col]
    .dropna()
    .astype(str).str.strip()
    .replace({"": "Unknown"})
    .value_counts()
)
if counts.empty or counts.sum() == 0:
    raise ValueError("No active-member country data to plot.")

total = counts.sum()
perc  = counts / total * 100.0

# Decide which slices get OUTSIDE labels (country + %)
if min_percent is not None:
    label_mask = perc >= float(min_percent)
elif top_n is not None:
    label_mask = counts.rank(method="first", ascending=False) <= int(top_n)
else:
    label_mask = counts.rank(method="first", ascending=False) <= 8

labeled_counts   = counts[label_mask].sort_values(ascending=False)
unlabeled_counts = counts[~label_mask]

# Group small ones into "Other" (single aggregate)
if group_other and not unlabeled_counts.empty:
    data_counts = pd.concat([labeled_counts, pd.Series({"Other": unlabeled_counts.sum()})])
else:
    data_counts = counts

data_perc = (data_counts / data_counts.sum()) * 100.0

# ---- Inside numbers: show member counts inside each wedge (non-bold)
def autopct_counts(pct):
    return f"{int(round(pct * data_counts.sum() / 100.0))}"

# Slight explode for the largest slices enhances depth
explode = [0.03 if (n in labeled_counts.index) or (n == "Other") else 0.0
           for n in data_counts.index]

# ---- Plot: FULL PIE, no global shadow (Fix B uses per-wedge patch shadows)
fig = plt.figure(figsize=(10, 10))
colors = plt.cm.Set3.colors

wedges, texts, autotexts = plt.pie(
    data_counts.values,
    labels=None,                      # we place outside labels manually
    autopct=autopct_counts,           # counts inside
    pctdistance=0.65,
    colors=colors,
    startangle=90,
    counterclock=False,
    explode=explode,
    shadow=False,                     # << NO big global shadow
    textprops=dict(fontsize=10, fontweight="normal", color="black"),
    wedgeprops=dict(linewidth=0.6, edgecolor="white")
)

# Inside counts in normal weight
for t in autotexts:
    t.set_fontweight("light")

# ---- Add a subtle per-wedge shadow 
#     Small offset + low alpha → depth without wide blooms when saving
for w in wedges:
    w.set_path_effects([
        pe.SimplePatchShadow(offset=(3, -3), shadow_rgbFace=(0, 0, 0, 0.15)),
        pe.Normal()
    ])

# ---- Outside labels: two texts so we can set different weights
country_fp = FontProperties(weight="normal")
percent_fp = FontProperties(weight="light")  # fallback if no 'light': FontProperties(weight=300)

# Which wedges get outside labels?
outside_mask = [(n == "Other") or (n in labeled_counts.index) for n in data_counts.index]

# ---- Connector geometry (straight elbow leaders)
r_edge   = 1.02   # contact point on wedge edge
r_elbow  = 1.12   # elbow radius (radial segment ends here)
r_text   = 1.26   # horizontal label anchor
hpad     = 0.06   # little gap so text doesn't sit on the line
dot_radius_offset = 0.015  # push dot slightly above the wedge
dot_size          = 18
dot_edgecolor     = "white"
dot_linewidth     = 0.8

# Connector styling
line_color  = "gray"
line_width  = 1.4
joinstyle   = "miter"  # crisp corner
capstyle    = "butt"   # square ends

ax = plt.gca()
ax.set_aspect('equal')

for w, name, pct, show in zip(wedges, data_counts.index, data_perc.values, outside_mask):
    if not show:
        continue

    # mid-angle of wedge
    theta = np.deg2rad((w.theta1 + w.theta2) / 2.0)
    ux, uy = np.cos(theta), np.sin(theta)

    # Points for the polyline connector: wedge → elbow → horizontal
    x0, y0 = r_edge * ux,  r_edge * uy         # wedge exit
    x1, y1 = r_elbow * ux, r_elbow * uy        # elbow point
    x2 = r_text if ux >= 0 else -r_text
    y2 = y1                                     # horizontal segment at elbow height
    ha = "left" if ux >= 0 else "right"

    # Single polyline path ensures perfectly straight elbow (no curvature)
    verts = [(x0, y0), (x1, y1), (x2, y2)]
    codes = [Path.MOVETO, Path.LINETO, Path.LINETO]
    connector = PathPatch(
        Path(verts, codes),
        facecolor="none",
        edgecolor=line_color,
        linewidth=line_width,
        joinstyle=joinstyle,
        capstyle=capstyle,
        zorder=6
    )
    ax.add_patch(connector)

    # Dot on the wedge end (match wedge color for polish)
    dot_color = w.get_facecolor()
    x_dot = (r_edge + dot_radius_offset) * ux
    y_dot = (r_edge + dot_radius_offset) * uy
    ax.scatter([x_dot], [y_dot], s=dot_size, color=dot_color,
               edgecolors=dot_edgecolor, linewidths=dot_linewidth, zorder=7)

    # Two separate texts so we can set different weights:
    x_text = x2 + (hpad if ha == "left" else -hpad)
    # small vertical offsets to stack lines neatly
    ax.text(x_text, y2 + 0.04, f"{name}",
            ha=ha, va="center", fontsize=10, color="black",
            fontproperties=country_fp, zorder=7)
    ax.text(x_text, y2 - 0.02, f"{pct:.1f}%",
            ha=ha, va="center", fontsize=10, color="black",
            fontproperties=percent_fp, zorder=7)

# ---- Final touches & (optional) save
plt.title(title, fontsize=14)
plt.tight_layout()

# Save with tight bbox to avoid clipping long leaders/labels
# (uncomment to save)
# fig.savefig("NASC_Active_Membership_by_Country_PIE_elbow_fixB.png",
#             dpi=300, bbox_inches="tight", facecolor="white")

plt.show()


In [ ]:
# save figure
if save_figs:   
    fig.savefig(
        f"NASC_{membership_toplot}_Membership_by_Country_PIE.png",
        dpi=300, bbox_inches="tight", facecolor="white"
    )


## Plot distribution of ACTIVE & CHAIR members by WG 

In [ ]:
# plot histogram with the number of ACTIVE_MEMBER per WG
plt.figure(figsize=(10, 6))

# get number of active members per WG: split by ";" and explode
df_active["ACTIVE_MEMBER"].str.split(";").explode().value_counts().plot(kind='bar', label=f"Active members ({total_active})", color="purple", alpha=0.5)
# print number of active members per WG
print(df_active["ACTIVE_MEMBER"].str.split(";").explode().value_counts())

# save number of active members per WG to a csv file
df_active["ACTIVE_MEMBER"].str.split(";").explode().value_counts().to_csv("NASC_active_members_by_SWG.csv", header=["Number of Active Members"])


# add in the same plot number of CHAIR_REQUESTED per WG
df_chair_req["CHAIR_REQUESTED"].str.split(";").explode().value_counts().plot(kind='bar', alpha=0.5, color='green', label=f"Chairs Requested ({total_chairs_req})")
print(df_chair_req["CHAIR_REQUESTED"].str.split(";").explode().value_counts())

# add informative box text with WG description in the top right corner
labels = df_clean0["ACTIVE_MEMBER"].str.split(";").explode().value_counts().index
# remove empty strings and leading/trailing spaces
labels = [x for x in labels if x]
labels = [x.strip() for x in labels]
#sort labels and get unique values
labels = list(set(labels))
labels.sort()
plt.text(0.55, 0.7, "\n".join(labels), fontsize=9, transform=plt.gcf().transFigure)

plt.xticks(rotation=90)
# set xlabel: "Working Group"
plt.xlabel("Working Group")
# set ylabel: "Number of active members"
plt.ylabel("Number of members")
plt.title("Active members by Working Group")

# add legend
plt.legend(loc="upper right", bbox_to_anchor=(0.5, 1.0))
# add the number of chairs as a text at the top of the red bars
for i, v in enumerate(df_chair_req["CHAIR_REQUESTED"].str.split(";").explode().value_counts()):
    plt.text(i, v + 1, str(v), color='green', ha='center')

# Save the plot to a file
if save_figs:
    plt.savefig("NASC_membership_by_WG.png", dpi=300, bbox_inches='tight')
plt.show()

## Plot Distribution By Gender

In [ ]:
# bar plot of requested chairs per "Gender"
plt.figure(figsize=(10, 6))

# support members: purple
df_support["Gender"].value_counts().plot(kind='bar', position=0.3,label=f'Support Members ({total_support})', color='blue', alpha=0.5, logy=True)
# add label with percentage of support members for each gender
for i, v in enumerate(df_support["Gender"].value_counts()):
	percent = v / total_support
	yloc = v + 5
	if percent < 0.05:
		yloc = v + 1
	plt.text(i+0.1, yloc, f"{v} ({percent:.1%})", ha='center', color='blue')

# active members:blue
df_active["Gender"].value_counts().plot(kind='bar', position=0.5, label=f'Active Members ({total_active})', color='purple',alpha=0.5, logy=True)
# add label with percentage of active members for each gender
total_active = len(df_active)
for i, v in enumerate(df_active["Gender"].value_counts()):
	percent = v / total_active
	yloc = v + 5
	if percent < 0.05:
	 	yloc = v + 1
	plt.text(i, yloc, f"{v} ({percent:.1%})", ha='center', color='purple')

# chairs:red
df_chair_req["Gender"].value_counts().plot(kind='bar',position=0.1, label=f'Requested Chairs ({total_chairs_req})', color="green", alpha=0.5, logy=True)
# add label with percentage of active members for each gender
for i, v in enumerate(df_chair_req["Gender"].value_counts()):
	percent = v / total_chairs_req
	color = 'lightgreen'
	yloc = v + 5
	if percent < 0.2:
		yloc = v + 0.4
	plt.text(i+0.2, yloc, f"{v} ({percent:.1%})", ha='center', color=color)


# get xticks values
xticks = plt.xticks()[0]
# get xticks labels (3rd element, once separated by commas)
xticklabels = plt.xticks()[1]
#separate by commas
xticklabels = [x.get_text() for x in xticklabels]
# replace empty ticks with "Unanswered" and "Gender diverse **" by "Gender diverse"
xticklabels = ["Unanswered" if x == "" else x for x in xticklabels]
xticklabels = ["Gender diverse" if x == "Gender diverse (gender non-conforming and/or transgender)" else x for x in xticklabels]
plt.xticks(xticks, xticklabels)
plt.title("Distribution by Gender")
# set xlabel
plt.xlabel("")
# set ylabel
plt.ylabel("Number of members")
plt.legend()
plt.tight_layout()
# Save the plot to a file
if save_figs:
	plt.savefig("NASC_membership_by_Gender.png", dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# Get total members per seniority: first merge "Mr", "Ms" and "Mx" in same category "Mr/Ms/Mx"
merged_name = "Pre-Doc"
title_mapping = {
    'Mr': merged_name,
    'Ms': merged_name,
    'Mx': merged_name,
    'Dr': 'Dr',
    'Prof.': 'Prof.'
}

# Apply the mapping to a new column
df_clean["MergedTitle"] = df_clean["Title"].map(title_mapping)
df_support = df_clean[df_clean["SUPPORT_MEMBER"] == "Yes"]
df_chair_req = df_clean[df_clean["CHAIR_REQUESTED"] != ""]
df_active = df_clean[df_clean["ACTIVE_MEMBER"] != ""]

all_titles = df_clean["MergedTitle"].value_counts()

# Sort titles by total members (descending)
sorted_titles = all_titles.sort_values(ascending=False).index.tolist()


# Build aligned counts
total = df_clean["MergedTitle"].value_counts().reindex(sorted_titles, fill_value=0)
active = df_active["MergedTitle"].value_counts().reindex(sorted_titles, fill_value=0)
support = df_support["MergedTitle"].value_counts().reindex(sorted_titles, fill_value=0)
chair_req = df_chair_req["MergedTitle"].value_counts().reindex(sorted_titles, fill_value=0)

# Combine into one DataFrame
counts_df = pd.DataFrame({
    f'Total members ({total_members})': total,
    f'Active members ({total_active})': active,
    f'Support members ({total_support})': support,
    f'Requested Chair ({total_chairs_req})': chair_req
})

bars_colors = ['red', 'purple', 'blue', 'green']

# Plot and add label to each barplot
ax = counts_df.plot(kind='bar', width=0.4, logy=True, figsize=(10, 6), color=bars_colors, alpha=0.5)
ax.set_xlabel("")
ax.set_ylabel("Number of Members")
ax.set_title("NASC Membership by Seniority")
plt.xticks(rotation=0)
plt.tight_layout()

# Save the plot to a file
if save_figs:
    plt.savefig("NASC_Membership_by_Seniority.png", dpi=300, bbox_inches='tight')
plt.show()

## Create list with the members of each WG

In [ ]:
# select members of each WG (including chairs)
for wg in ["WG1", "WG2", "WG3", "WG4", "WG5", "WG6", "WG7"]:
    # get members of each WG (being active or chair)
    df_wg = df_clean[df_clean["ACTIVE_MEMBER"].str.contains(wg, na=False) | df_clean["CHAIR_REQUESTED"].str.contains(wg, na=False)]
    # print members of each WG
    #print(f"Members of {wg}:")
    #print(df_wg[["Name", "Country1", "ACTIVE_MEMBER", "EMAIL"]])
    # save to file
    #df_wg.to_excel(f"{wg}_members_{date}.xlsx", index=False)



## Cross-match with members of the repository

Using data download form UniGe repository, check how many 'old' members are now in the new NASC community

In [ ]:
xml_repo = "j2xml2112020250625082623.xml"
# Load the XML file into a DataFrame
df_repo = pd.read_xml(xml_repo)
print(df_repo.columns)

#extract columns of interest: name, email
df_repo = df_repo[["name", "email"]]
#print(df_repo.head())
# remove rows with empty 'name' or 'email' columns or with None values
df_repo = df_repo.dropna(subset=['name', 'email'])
df_repo = df_repo[(df_repo['name'] != '') & (df_repo['email'] != '') & (df_repo['name'] != 'None')]

# extract surname from name
surname_prefixes = ['van', 'von', 'de', 'den', 'del', 'della', 'di', 'la', 'le', 'du', 'da']
pattern = re.compile(r'\b(?:' + '|'.join(surname_prefixes) + r')\b(?:\s+\b\w+\b)+|\b\w+$', re.IGNORECASE)
# Function to extract surname
def extract_surname(name):
    match = pattern.search(name)
    return match.group(0) if match else None
df_repo['surname'] = df_repo['name'].apply(extract_surname)

# remove leading and trailing spaces in 'name' column
df_repo['surname'] = df_repo['surname'].str.strip()
# change surname to lowercase
df_repo['surname'] = df_repo['surname'].str.lower()
# change 'email' to lower case
df_repo['email_lower'] = df_repo['email'].str.lower()
# remove leading and trailing spaces in 'email' column
df_repo['email_lower'] = df_repo['email_lower'].str.strip()
print(df_repo)


In [ ]:
# cross-match with members of the df_clean DataFrame
# select only interesting columns from df_clean: 'Name', 'EMAIL'
df_active_for_repo = df_active[['Name', 'Last Name', 'EMAIL', 'ACTIVE_MEMBER']].copy()
# change 'Last Name' to lowercase
df_active_for_repo['LastName_lower'] = df_active_for_repo['Last Name'].str.lower()
# remove leading and trailing spaces in 'Last Name' column
df_active_for_repo['LastName_lower'] = df_active_for_repo['LastName_lower'].str.strip()
# change 'EMAIL' to lower case
df_active_for_repo['EMAIL_lower'] = df_active_for_repo['EMAIL'].str.lower()
# remove leading and trailing spaces in 'EMAIL' column
df_active_for_repo['EMAIL_lower'] = df_active_for_repo['EMAIL_lower'].str.strip()

print(df_active_for_repo.columns)
print(df_repo.columns)

# cross-match members with email coincidence 
cross_match = df_active_for_repo.merge(df_repo, left_on=['EMAIL_lower'], right_on=['email_lower'], how='inner')
# open file to save cross-matched members
cross_match_tofile = cross_match.copy()
cross_match_tofile = cross_match_tofile.drop(columns=['Last Name','LastName_lower', 'surname', 'email_lower', 'EMAIL_lower'])
cross_match_tofile.rename(columns={'Name': 'Name_NASC', 'EMAIL': 'EMAIL_NASC', 'name':'Name_REPO','email': 'email_REPO'}, inplace=True)
#cross_match_tofile.to_excel(f"cross_matched_members_NASC_repo.xlsx", index=False)
print(f"Cross-matched members (by email): {len(cross_match)}")
print("=====================================================")
# for each cross-matched member, print entry in df_clean and df_repo
for index, row in cross_match.iterrows():
    #print(f"Cross-matched member: {row['Name']}")
    entry_NASC = df_active_for_repo[df_active_for_repo['EMAIL_lower'] == row['EMAIL_lower']].to_dict(orient='records')
    entry_repo = df_repo[df_repo['email_lower'] == row['EMAIL_lower']].to_dict(orient='records')
    #print(f"    Entry in NASC: {entry_NASC}")
    #print(f"    Entry in REPO: {entry_repo}")
# remove entry_NASC and entry_repo from df_active_for_repo and df_repo
df_active_for_repo_remaining = df_active_for_repo[~df_active_for_repo.set_index(['EMAIL_lower']).index.isin(cross_match.set_index(['EMAIL_lower']).index)]
df_repo_remaining = df_repo[~df_repo.set_index(['email_lower']).index.isin(cross_match.set_index(['email_lower']).index)] 


# get cross-matched members with name coincidence but not email coincidence
cross_match_name_only = df_active_for_repo_remaining.merge(df_repo_remaining, left_on='LastName_lower', right_on='surname', how='inner')
cross_match_name_only = cross_match_name_only[cross_match_name_only['EMAIL_lower'] != cross_match_name_only['email_lower']]
print(f"Cross-matched members (Name only): {len(cross_match_name_only)}")
print("=====================================================")
#print(cross_match_name_only)
# for each cross-matched member, print entry in df_clean and df_repo
for index, row in cross_match_name_only.iterrows():
    #print(f"Cross-matched member (Name only): {row['Name']}")
    entry_NASC = df_active_for_repo_remaining[df_active_for_repo_remaining['LastName_lower'] == row['LastName_lower']].to_dict(orient='records')
    entry_repo = df_repo_remaining[df_repo_remaining['surname'] == row['LastName_lower']].to_dict(orient='records')
    #print(f"    Entry in NASC: {entry_NASC}")
    #print(f"    Entry in REPO: {entry_repo}")
cross_match_name_only_tofile = cross_match_name_only.copy()
# drop columns LasName_lower and surname
cross_match_name_only_tofile = cross_match_name_only_tofile.drop(columns=['Last Name','LastName_lower', 'surname', 'email_lower', 'EMAIL_lower'])
# rename 'Name' to 'Name_NASC', 'EMAIL' to EMAIL_NASC and 'email' to 'email_REPO'
cross_match_name_only_tofile.rename(columns={'Name': 'Name_NASC', 'EMAIL': 'EMAIL_NASC', 'name':'Name_REPO','email': 'email_REPO'}, inplace=True)
cross_match_name_only_tofile.to_excel(f"cross_matched_members_NASC_repo_change_email.xlsx", index=False)
df_active_for_repo_remaining_remaining = df_active_for_repo_remaining[~df_active_for_repo_remaining.set_index(['LastName_lower']).index.isin(cross_match_name_only.set_index(['LastName_lower']).index)]
df_repo_remaining_remaining = df_repo_remaining[~df_repo_remaining.set_index(['surname']).index.isin(cross_match_name_only.set_index(['surname']).index)] 

# save remaining members to a file
df_active_for_repo_remaining_remaining_tofile = df_active_for_repo_remaining_remaining.copy()
df_active_for_repo_remaining_remaining_tofile = df_active_for_repo_remaining_remaining_tofile.drop(columns=['Last Name','LastName_lower', 'EMAIL_lower'])
df_active_for_repo_remaining_remaining_tofile.rename(columns={'Name': 'Name_NASC', 'EMAIL': 'EMAIL_NASC', 'name':'Name_REPO','email': 'email_REPO'}, inplace=True)
df_repo_remaining_remaining_tofile = df_repo_remaining_remaining.copy()
df_repo_remaining_remaining_tofile = df_repo_remaining_remaining_tofile.drop(columns=['surname', 'email_lower'])
df_repo_remaining_remaining_tofile.rename(columns={'name': 'Name_REPO', 'email': 'email_REPO'}, inplace=True)

df_active_for_repo_remaining_remaining_tofile.to_excel(f"remaining_members_NASC_notREPO.xlsx", index=False)
df_repo_remaining_remaining_tofile.to_excel(f"remaining_members_REPO_notNASC.xlsx", index=False)


In [ ]:
# check particular cases
surname = "den Herder"
surname_lower = surname.lower()

# look for it in df_clean
df_check = df_active_for_repo_remaining_remaining[df_active_for_repo_remaining_remaining['LastName_lower'] == surname_lower]
print(f"Members with required LastName_lower '{surname}': {len(df_check)}")
print(df_check[['Name', 'LastName_lower', 'EMAIL']])
# look for it in df_repo
df_check_repo = df_repo_remaining_remaining[df_repo_remaining_remaining['surname'] == surname_lower]
print(f"Members with required surname '{surname}': {len(df_check_repo)}")
print(df_check_repo[['name', 'surname', 'email']])

## Matrices   
Extra work (To be reviewed with new registrations)

In [ ]:


# keep columns: ID, WG0, WG1, WG2
df = df[["ID", "WG0", "WG1", "WG2"]]
# Rename column WG0 to "Primary WG"
df.rename(columns={"WG0": "Primary WG"}, inplace=True)
# Rename column WG1 to "Secondary WG"
df.rename(columns={"WG1": "Secondary WG"}, inplace=True)
# Rename column WG2 to "Tertiary WG"
df.rename(columns={"WG2": "Tertiary WG"}, inplace=True)
# Replace cells content in these columns:
# "WG1: Large-scale structure of the Universe" -> "WG1"
# "WG2: Galaxies and supermassive black holes" -> "WG2"
# "WG3: Stars and their environment" -> "WG3"
# "WG4: Compact objects" -> "WG4"
# "WG5: Transients and multi-messenger astrophysics" -> "WG5"
# "WG6: Cosmology and fundamental physics" -> "WG6"
# "WG7: Science Support" -> "WG7"

# Strip leading and trailing whitespaces or tabs
df["Primary WG"] = df["Primary WG"].str.strip()
df["Secondary WG"] = df["Secondary WG"].str.strip()
df["Tertiary WG"] = df["Tertiary WG"].str.strip()
# in each column replace the content of the cell with the first 3 letters of the cell content
df["Primary WG"] = df["Primary WG"].str[:3]
df["Secondary WG"] = df["Secondary WG"].str[:3]
df["Tertiary WG"] = df["Tertiary WG"].str[:3]

# remove rows with empty cells in all the groups
df = df.dropna(subset=["Primary WG", "Secondary WG", "Tertiary WG"], how="all")

# keep only the rows with an empty cell in the "Tertiary WG" column
df = df[df["Tertiary WG"].isnull()]
# remove the "Tertiary WG" column
df = df.drop(columns=["Tertiary WG"])
# remove rows with empty cells in all the groups
df = df.dropna(subset=["Primary WG", "Secondary WG"], how="all")


In [ ]:
# salva el dataframe en un archivo CSV
df.to_csv("membership_clean2.csv", index=False)

In [ ]:
# Definir los nombres de las columnas
df = pd.read_csv("membership_clean2.csv", header=0)
col1 = "ID"
col2 = "Primary WG"
col3 = "Secondary WG"

# Reemplazar valores NaN con strings vacíos y asegurarse de que son strings
df = df.fillna("")

# Obtener la lista única de WGs excluyendo valores vacíos
wgs = sorted(set(df[col2]).union(df[col3]) - {""})

# Crear una matriz de ceros de tamaño (n_WGs x n_WGs)
matrix = pd.DataFrame(np.zeros((len(wgs), len(wgs))), index=wgs, columns=wgs)

# Llenar la matriz contando las asociaciones
for _, row in df.iterrows():
    selected_wgs = {row[col2], row[col3]} - {""}  # Filtrar valores vacíos
    
     # Llenar la diagonal con personas que solo pertenecen a un WG
    if len(selected_wgs) == 1:
        wg = list(selected_wgs)[0]
        matrix.loc[wg, wg] += 1  # Contar personas que solo eligieron este WG
    
    # no contar dos veces la misma asociación
    elif len(selected_wgs) == 2:
        wg1, wg2 = selected_wgs
        matrix.loc[wg1, wg2] += 1
        matrix.loc[wg2, wg1] += 1
    

# Graficar el heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(matrix, annot=True, cmap="Blues", linewidths=0.5, fmt=".0f", cbar=True)

# Configuración del gráfico
plt.title("WGs association heatmap")
plt.xlabel("WG")
plt.ylabel("WG")
plt.xticks(rotation=45)
plt.yticks(rotation=0)

# Mostrar
plt.show()
# Guardar la figura
plt.savefig("heatmap2.png")
#cerrar la figura
plt.close()



In [ ]:
# Crear un grafo vacio
G = nx.Graph()
# Agregar los nodos al grafo: 
# usa solo las columnas "Primary WG", "Secondary WG". No uses "Tertiary WG"
G.add_nodes_from(df[["Primary WG", "Secondary WG"]].dropna().values.flatten())

# Agregar las aristas al grafo: solo si los valores no son None
# usa solo las columnas "Primary WG", "Secondary WG". No uses "Tertiary WG"
G.add_edges_from(df[["Primary WG", "Secondary WG"]].dropna().values)
# Dibujar el grafo: haz los nodos mas grandes y usa el layout "spring" y evita mucho espacio en blanco
plt.figure(figsize=(10, 10))
nx.draw(G, node_size=5000, pos=nx.spring_layout(G), with_labels=True)
# add text box with the WGs names
# "WG1: Large-scale structure of the Universe"
# "WG2: Galaxies and supermassive black holes"
# "WG3: Stars and their environment"
# "WG4: Compact objects"
# "WG5: Transients and multi-messenger astrophysics"
# "WG6: Cosmology and fundamental physics"
# "WG7: Science Support"
fsize = 10
plt.text(0.5, 0.5, "WG1: Large-scale structure of the Universe", fontsize=fsize, ha='center')
plt.text(0.5, 0.45, "WG2: Galaxies and supermassive black holes", fontsize=fsize, ha='center')
plt.text(0.5, 0.4, "WG3: Stars and their environment", fontsize=fsize, ha='center')
plt.text(0.5, 0.35, "WG4: Compact objects", fontsize=fsize, ha='center')
plt.text(0.5, 0.3, "WG5: Transients and multi-messenger astrophysics", fontsize=fsize, ha='center')
plt.text(0.5, 0.25, "WG6: Cosmology and fundamental physics", fontsize=fsize, ha='center')
plt.text(0.5, 0.2, "WG7: Science Support", fontsize=fsize, ha='center')
# Guardar la figura en un archivo
plt.savefig("graph.png")
# Mostrar la figura
plt.show()

In [ ]:
# localiza las líneas donde aparece WG7 en alguna de las columnas
df.loc[(df["Primary WG"] == "WG7") | (df["Secondary WG"] == "WG7") | (df["Tertiary WG"] == "WG7")]
# cuántos son?
len(df.loc[(df["Primary WG"] == "WG7") | (df["Secondary WG"] == "WG7") | (df["Tertiary WG"] == "WG7")])

# localiza las líneas donde aparece WG7 en más de una columna
df.loc[(df["Primary WG"] == "WG7") & (df["Secondary WG"] == "WG7") | (df["Primary WG"] == "WG7") & (df["Tertiary WG"] == "WG7") | (df["Secondary WG"] == "WG7") & (df["Tertiary WG"] == "WG7")]
